<a href="https://colab.research.google.com/github/brgama/fine-tuning-rag-documentos-fiap/blob/main/TechChallenge_Fase3_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Carregando arquivos do Gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" # Unsloth - biblioteca otimizada para treinar modelos de linguagem grandes (LLMs) de forma mais rápida e eficiente
!pip install triton # triton - compilador da NVIDIA que otimiza operações de deep learning, acelerando o treino de modelos LLM.
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes # Fine-tuning com LoRA, quantização e otimização de modelos
!pip install transformers datasets # Biblioteca da Hugging Face para carregar e treinar modelos como LLaMA, GPT e outros.

In [ ]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # Biblioteca principal do PyTorch para deep learning
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported # Método otimizado para carregar e treinar modelos LLMs de forma eficiente com Unsloth.

In [ ]:
# importando bibliotecas essenciais para processamento de dados e fine-tuning de um modelo de linguagem (LLM)

import json # Permite carregar e manipular arquivos JSON, que geralmente contêm dados estruturados.
import pandas as pd # Pandas é uma biblioteca para manipulação e análise de dados tabulares
from datasets import load_dataset #Permite carregar conjuntos de dados da Hugging Face ou arquivos locais.
from trl import SFTTrainer # classe usada para treinamento supervisionado de LLMs
from transformers import TrainingArguments # define parâmetros do treinamento
from transformers import pipeline # Fornece uma interface simples para usar modelos treinados

In [ ]:
# define configurações para carregar e treinar um modelo de linguagem (LLM)
max_seq_length = 512 # Define o comprimento máximo de sequência que o modelo pode processar
dtype = None # significa que o tipo será definido automaticamente com base na configuração do hardware
load_in_4bit = True # Ativa a quantização em 4 bits, reduzindo o uso de memória do modelo

In [ ]:
# define uma lista de modelos de linguagem compatíveis com Unsloth e quantização em 4 bits
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


# Resumo
# Define uma lista de modelos de linguagem compatíveis com Unsloth.
# Inclui Mistral, LLaMA-3, Phi-3 e Gemma, otimizados para 4-bit (uso reduzido de VRAM).
# Pode ser usada para escolher dinamicamente um modelo para inferência ou fine-tuning


In [ ]:
# Função para formatar o dataset

def format_dataset_into_model_input(data):
    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Extraindo o título e a descrição
    for example in data:
        title = example['title']
        content = example['content']
        instruction = "Generate a persuasive product description that highlights key features, benefits, and unique selling points in an engaging and clear manner."

        instructions.append(instruction)
        inputs.append(title)
        outputs.append(content)

    # Criando o dicionário com os dados formatados
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando os dados formatados em um arquivo JSON (GDRIVE)
    formatted_json_path = "/content/drive/MyDrive/FIAP/amazon_file_formatted.json"
    with open(formatted_json_path, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset formatado salvo em: {formatted_json_path}")

    return formatted_json_path

In [ ]:
DATA_PATH = '/content/drive/MyDrive/FIAP/trn.json' # Caminho contendo o dataset original
dataset = load_dataset("json", data_files=DATA_PATH) # Usa a função load_dataset() da biblioteca Hugging Face Datasets para carregar os dados do arquivo JSON.
OUTPUT_PATH_DATASET = format_dataset_into_model_input(dataset['train'] if 'train' in dataset else dataset) # Verifica se o dataset tem uma divisão "train", formata os dados para fine-tuning e salva os dados formatados em um novo arquivo JSON.

Dataset formatado salvo em: /content/drive/MyDrive/FIAP/amazon_file_formatted.json


In [ ]:
# Carregando o Modelo Pré-treinado
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)


# Resumo
# Carrega o modelo Mistral 7B quantizado em 4 bits para otimizar desempenho e memória.
# Utiliza a biblioteca Unsloth, que melhora a eficiência do modelo na GPU.
# Retorna o modelo e o tokenizer, permitindo inferência ou treinamento.

==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 7.5. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Preparando o Modelo para inferência, otimizando a sua performance para fazer previsões
FastLanguageModel.for_inference(model)


# Resumo
# A inferência serve para gerar resultados a partir de um modelo treinado, como respostas, predições ou textos baseados em um prompt de entrada.
# A função for_inference(model) prepara o modelo para esse processo, garantindo que ele seja executado da forma mais eficiente possível.
#Isso é especialmente útil em aplicações de chatbots, assistentes virtuais, geração de conteúdo e análise de texto em tempo real.

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): M

In [ ]:
def generate_description(title):
    input_text = f"Generate a persuasive product description that highlights key features, benefits, and unique selling points in an engaging and clear manner.\nTítulo: {title}\nDescrição:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

# Resumo
# Criação de um prompt de entrada específico para gerar descrições.
# Tokenização e movimento para GPU para otimizar a execução.
# Geração de texto e decodificação da resposta para obter uma descrição legível.

In [ ]:
title = "Smartphone Samsung Android 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

#  Resumo:
# O código passa o título de um produto ("Smartphone Samsung Android 64GB") para a função generate_description(), que gera uma descrição persuasiva e detalhada do produto.
# O título e a descrição gerada são então impressos no console.

Título: Smartphone Samsung Android 64GB
Descrição: Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone Samsung Android 64GB
Descrição:

The Samsung Galaxy S20 FE is a powerful smartphone that offers a range of features and capabilities to enhance your mobile experience. With its 6.5-inch Super AMOLED display, you can enjoy vibrant and clear visuals for your favorite movies, games, and more. The phone is powered by a Qualcomm Snapdragon 865 processor, ensuring smooth and efficient performance.

The Galaxy S20 FE features a triple-lens camera system, including a 12MP ultra-wide lens, a 12MP wide-angle lens, and an 8MP


In [ ]:
title = "Smartphone iphone 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

#  Resumo:
# O código passa o título de um produto ("Smartphone Samsung Android 64GB") para a função generate_description(), que gera uma descrição persuasiva e detalhada do produto.
# O título e a descrição gerada são então impressos no console.

Título: Smartphone iphone 64GB
Descrição: Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone iphone 64GB
Descrição:

The iPhone 64GB is a powerful and versatile smartphone that offers a range of features and capabilities to enhance your daily life. With its sleek design, advanced technology, and intuitive user interface, the iPhone 64GB is a top choice for those who demand the best in mobile technology.

One of the standout features of the iPhone 64GB is its large, high-resolution display. The 6.1-inch Super Retina XDR display offers stunning visuals with a resolution of 2532 x 1170 pixels, making it perfect for


In [ ]:
# Ajustes finos do LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # Aumentado para melhorar a capacidade de ajuste
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,  # Aumentado para ajustar melhor os parâmetros
    lora_dropout=0.1,  # Introduzido dropout para evitar overfitting
    bias="none", # Não utiliza viés durante o ajuste
    use_gradient_checkpointing="unsloth",
    random_state=3407, # Estado aleatório para reprodutibilidade
    use_rslora=False, # Não usa RSLora (uma variante do LoRA)
    loftq_config=None, # Configuração específica de loftq não usada
)


# Resumo:
# Este código aplica ajustes finos (fine-tuning) no modelo de linguagem usando a técnica LoRA.
# Ele ajusta de forma eficiente módulos específicos do modelo relacionados à atenção, sem a necessidade de treinar o modelo inteiro.
# Os parâmetros configurados visam otimizar a capacidade de ajuste (com r e lora_alpha), evitar overfitting (com lora_dropout), e permitir um treinamento mais eficiente (com gradient_checkpointing).
# Esses ajustes finos permitem adaptar o modelo para tarefas específicas, utilizando menos recursos computacionais e evitando o treinamento completo do modelo.

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
# Prompt para a descrição dos Produtos
new_prompt = """Here is a task description, along with a title that offers more context. Please write a response that effectively completes the description.

### Instrução:
{}

### Título:
{}

### Descrição:
{}"""

EOS_TOKEN = tokenizer.eos_token

# Resumo:
# O código define um template de prompt estruturado para gerar descrições de produtos. Ele inclui uma instrução sobre o que o modelo deve fazer, um título do produto e um campo para a descrição gerada.
# A variável EOS_TOKEN armazena o token de fim de sequência do modelo, utilizado para finalizar a geração de texto.
#Esse tipo de template ajuda a garantir que o modelo entenda o contexto e a tarefa a ser realizada de maneira mais clara e estruturada.


In [ ]:
# Função para formatar o Prompt

def formatting_prompts_func(data):
    instructions = data["instruction"]
    inputs = data["input"]
    outputs = data["output"]

    texts = []

    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = new_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

# A função formata os dados extraídos de data e os organiza em prompts estruturados que podem ser utilizados para treinar ou gerar respostas com um modelo de linguagem.

In [ ]:
# Carregar o dataset
dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET)

print(dataset)
print(dataset['train'].column_names)

# Resumo:
# O código carrega um dataset de um arquivo JSON localizado em OUTPUT_PATH_DATASET.
#Em seguida, ele imprime o dataset carregado (mostrando suas divisões e informações) e os nomes das colunas da parte train desse dataset. Isso ajuda a verificar a estrutura dos dados e as colunas presentes no conjunto de treinamento.

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 2248619
    })
})
['instruction', 'input', 'output']


In [ ]:
# Verificar uma amostra
print(dataset['train'][0])  # Verifique uma amostra do dataset

{'instruction': 'Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.', 'input': 'Girls Ballet Tutu Neon Pink', 'output': 'High quality 3 layer ballet tutu. 12 inches in length'}


In [ ]:
# Formatando os dados
formatted_dataset = dataset['train'].map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/2248619 [00:00<?, ? examples/s]

In [ ]:
# dataset resultante após a aplicação da função de formatação
formatted_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 2248619
})

In [ ]:
dataset['train'][0]

{'instruction': 'Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.',
 'input': 'Girls Ballet Tutu Neon Pink',
 'output': 'High quality 3 layer ballet tutu. 12 inches in length'}

In [ ]:
#Configurando o Trainer para o modelo de aprendizado

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,  # Aumento dos warmup steps para melhorar a estabilidade do treinamento
        max_steps=60,  # Aumento no número de passos de treinamento
        learning_rate=1e-4,  # Taxa de aprendizado ajustada para um ajuste mais refinado
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,  # Log mais frequente para monitorar o treinamento
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/2248619 [00:00<?, ? examples/s]

In [ ]:
# Iniciando o processo de treinamento do modelo
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,248,619 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080/3,842,248,704 (2.18% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: brunogamarj (brunogamarj-yduqs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.408200
20,1.892100
30,1.317400
40,1.279200
50,1.349700
60,1.232700


In [ ]:
# Salvar o modelo ajustado (GDRIVE)
model.save_pretrained("/content/drive/MyDrive/FIAP/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/FIAP/lora_model")

('/content/drive/MyDrive/FIAP/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/FIAP/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/FIAP/lora_model/tokenizer.model',
 '/content/drive/MyDrive/FIAP/lora_model/added_tokens.json',
 '/content/drive/MyDrive/FIAP/lora_model/tokenizer.json')

In [ ]:
model_path = "/content/drive/MyDrive/FIAP/lora_model" # Caminho para o modelo salvo
model, tokenizer = FastLanguageModel.from_pretrained(model_path) # Carregar o modelo e o tokenizer
FastLanguageModel.for_inference(model) # Preparar o modelo para inferência

==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 7.5. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/drive/MyDrive/FIAP/lora_model as a legacy tokenizer.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096, padding_idx=770)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
    

In [ ]:
# Verificando a resposta
title = "Smartphone Samsung Android 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Smartphone Samsung Android 64GB
Descrição: Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone Samsung Android 64GB
Descrição:

### Product Description

Samsung Galaxy S6 Edge 64GB

### From the Manufacturer

##### Samsung Galaxy S6 Edge 64GB

The Samsung Galaxy S6 Edge is a smartphone that is designed to be a work of art. The Galaxy S6 Edge is the first smartphone to feature an edge-to-edge display, which is curved on both sides. The curved display is not only beautiful, but it also provides a more comfortable grip and easier access to the phone's features. The Galaxy S6 Edge is also the first smartphone to feature


In [ ]:
# Verificando a resposta
title = "Smartphone iphone 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Smartphone iphone 64GB
Descrição: Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone iphone 64GB
Descrição:

### Product Description

iPhone 6 Plus is an elegant, advanced smartphone that's thinner and lighter than iPhone 5s. It features a 5.5-inch Retina HD display with 1920-by-1080 resolution, 1.4 million pixels, and 401 ppi. iPhone 6 Plus is powered by the A8 chip with second-generation 64-bit desktop-class architecture, and an M8 motion coprocessor. It features a 8MP iSight camera with 1.5µ pixels
